# 0 - Librerías y variables

In [6]:
# Librerías
# ------------------------------------------------------------------------------
import os
import requests
import json

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [7]:
# Variables
# ------------------------------------------------------------------------------
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1 - LLM Chat

En este apartado se llama a la API de OpenAI directamente:

In [8]:
# Importar OpenAI
from openai import OpenAI

# Inicializar el cliente
client = OpenAI()

# Llamar al LLM
messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]
    
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)

response.choices[0].message.content

'1+1 es igual a 2.'

Si quiero modificar el código por ejemplo para utilizar un modelo como LLaMA 3.1 desde Ollama:

In [9]:
# Define el endpoint local de Ollama
OLLAMA_URL = "http://localhost:11434/api/chat"

# Llamar al LLM
messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]

response = requests.post(OLLAMA_URL, json={
    "model": "llama3.1",
    "messages": messages,
    "temperature": 0.0,
    "stream": False
})

data = response.json()
print(data["message"]["content"])

La respuesta a la pregunta es 2.


Cada vez que quiero apuntar a un LLM diferente, tengo que modificar el código de forma sustancial.

# 2 - LangChain

LangChain es más un framework para construir sistemas que usan modelos de lenguaje (LLMs) que una simple librería, a continuación se muestra una tabla con los bloques principales del framework:

| **Bloque**                       | **Para qué sirve**                                                                           |
| -------------------------------- | -------------------------------------------------------------------------------------------- |
| **1. Modelos**                   | Generan texto o representaciones vectoriales (embeddings) a partir de texto                  |
| **2. Prompts**                   | Construyen entradas reutilizables, seguras y controladas para los modelos                    |
| **3. Output Parsers**            | Transforman la salida del modelo (texto) en estructuras útiles como JSON, listas u objetos   |
| **4. Memorias**                  | Guardan el historial o contexto entre interacciones (conversacionales o no)                  |
| **5. Tools y Agents**            | Ejecutan funciones externas y permiten que un agente decida dinámicamente qué hacer y cuándo |
| **6. Runnables**                 | Unifican cualquier componente ejecutable en un flujo modular y componible                    |
| **7. Retrievers y VectorStores** | Permiten búsquedas semánticas para recuperar información relevante desde grandes corpus      |
| **8. Chains**                    | Encadenan pasos fijos de procesamiento en flujos controlados (prompt → modelo → parseo)      |
| **9. Callbacks y Tracing**       | Monitorizan, trazan y depuran la ejecución para mejorar observabilidad y debugging           |
| **10. Loaders y Splitters**      | Cargan y fragmentan documentos largos para su posterior análisis o búsqueda                  |

## 2.1. - Modelos

En LangChain, los models (modelos) son los componentes que generan texto o responden a mensajes. Son la parte que realmente interactúa con modelos de lenguaje (LLMs) como los de OpenAI, Anthropic, Cohere, etc. LangChain organiza los modelos según lo que hacen. Los más comunes son:

| Tipo de modelo     | Qué hace                                                                    | Clase típica                                |
| ------------------ | --------------------------------------------------------------------------- | ------------------------------------------- |
| **LLM**            | Genera texto a partir de un *prompt plano*                                  | OpenAI, HuggingFaceHub                      |
| **ChatModel**      | Maneja conversaciones con roles (usuario, asistente, sistema)               | ChatOpenAI, ChatAnthropic                   |
| **EmbeddingModel** | Convierte texto en vectores (útiles para búsquedas o comparación semántica) | OpenAIEmbeddings, HuggingFaceEmbeddings     |


LangChain permite llamar a diferentes modelos, con diferentes APIs, de forma agnóstica. Cambiando ligeramente el código, puedo aprovechar una estructura ya creada para apuntar a otro modelo.

In [10]:
import langchain
print(langchain.__version__)

0.3.25


In [11]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [12]:
# Crear los mensajes en el formato de LangChain
messages = [
    HumanMessage(content="¿Cuánto es 1+1?")
]

In [13]:
# Llamar a un modelo A
llm = ChatOllama(
    model="llama3.1",
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

La respuesta a la pregunta de "¿Cuánto es 1+1?" es 2.


In [14]:
# Llamar a un modelo B
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

1+1 es igual a 2.


La estructura es exactamente la misma, se podría incluso encapsular el codigo en una función y pasar el modelo como parámetro.

In [15]:
def call_llm_model(
    model_name: str,
    temperature: float,
    message: str
) -> str:
    
    if model_name.startswith("gpt"):
        llm = ChatOpenAI(
            model_name=model_name,
            temperature=temperature
        )
    else:
        llm = ChatOllama(
            model=model_name,
            temperature=temperature
        )

    response = llm.invoke([HumanMessage(content=message)])
    return response.content


# Ejemplo de uso:
respuesta = call_llm_model(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    message="¿Quién es Andrew Ng?"
)

print(respuesta)

Andrew Ng es un científico de la computación e investigador en inteligencia artificial. Es conocido por su trabajo pionero en el campo del aprendizaje profundo y por ser uno de los cofundadores de Google Brain, así como de Coursera. Ng también ha sido profesor en la Universidad de Stanford y en la Universidad de California en Berkeley. Es una figura destacada en el campo de la inteligencia artificial y ha realizado importantes contribuciones en áreas como el reconocimiento de voz, la visión por computadora y la robótica.


En las últimas versiones de LangChain, se ha creado la abstracción `init_chat_model()` para inicilizar modelos de diferentes proveedores desde la misma función.

In [16]:
from langchain.chat_models import init_chat_model

# model = init_chat_model("gpt-4o-mini", model_provider="openai")
# model = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
# model = init_chat_model("mistral-large-latest", model_provider="mistralai")

## 2.2. - Prompt Templates

Las Prompt Templates (plantillas de prompt) son una herramienta de LangChain que te ayuda a crear automáticamente los mensajes que le envías al modelo de lenguaje, de una forma organizada, flexible y reutilizable.

Cuando trabajas con modelos como GPT, no sueles enviar directamente el texto del usuario al modelo. Normalmente quieres hacer algo más, como:

- Agregar instrucciones específicas (ej.: "Traduce al francés...")
- Dar un contexto adicional al modelo (ej.: "Eres un traductor especializado en literatura clásica...")
- Formatear el texto de forma especial (ej.: "Devuelve el resultado en un .json con la estructura...")
- Usar el mismo formato muchas veces con diferentes datos

Ahí es donde entran las prompt templates.

Piensa en ellas como plantillas con huecos (como los de un formulario). Tú defines una estructura fija y dejas espacios para completar con datos reales más tarde.

In [17]:
template = "Traduce el siguiente texto al {language}: {text}"

LangChain te permite definir esta plantilla y luego rellenarla automáticamente con los valores que quieras:
- `language = "francés"`
- `text = "Hola, ¿cómo estás?"`

Los principales templates y su propósito son:

| Prompt Template                    | ¿Para qué sirve?                                                                                                                                   |
| ---------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- |
| `ChatPromptTemplate`               | Plantilla para **modelos de chat**. Permite combinar mensajes (`System`, `Human`, etc.) fácilmente. Ideal para `ChatOpenAI`, `ChatAnthropic`, etc. |
| `PromptTemplate`                   | Plantilla simple de texto plano. Usado con modelos de lenguaje **no conversacionales** como `OpenAI(model="text-davinci-003")`.                    |
| `SystemMessagePromptTemplate`      | Subplantilla usada dentro de `ChatPromptTemplate` para definir el **mensaje del sistema**.                                                         |
| `HumanMessagePromptTemplate`       | Subplantilla usada dentro de `ChatPromptTemplate` para el **mensaje del usuario**                                                                  |
| `MessagesPlaceholder`              | Placeholder especial dentro de `ChatPromptTemplate` para insertar una lista dinámica de mensajes (ej: historial de conversación).                  |
| `AIMessagePromptTemplate`          | Subplantilla para simular un mensaje **anterior del asistente** en el historial.                                                                   |
| `FewShotPromptTemplate`            | Plantilla para tareas de **few-shot learning**. Permite definir ejemplos que se combinan con el input.                                             |
| `ChatMessagePromptTemplate`        | Versión más flexible que permite definir mensajes de un rol personalizado (ej: `"role": "function"`).                                              |
| `FewShotChatMessagePromptTemplate` | Lo mismo que `FewShotPromptTemplate`, pero adaptado para `ChatPromptTemplate`. Útil si quieres ejemplos en formato chat.                           |

In [18]:
from langchain_core.prompts import ChatPromptTemplate


| Método            | ¿Qué construye?                             | ¿Cuándo usarlo?                                 |
| ----------------- | ------------------------------------------- | ----------------------------------------------- |
| `from_template()` | Una sola plantilla de mensaje               | Para casos simples o rápidos                    |
| `from_messages()` | Una conversación entera con múltiples roles | Cuando necesitas dar contexto o varios mensajes |


In [19]:
# Plantilla de traducción
template = "Traduce el siguiente texto al {language}: {text}"

In [20]:
# Para casos sencillos
template_string = "Traduce el siguiente texto al {language}: {text}"
prompt_template = ChatPromptTemplate.from_template(template_string)

In [21]:
# Para casos mas complejos: sistema, usuarios, asistente...etc
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un trabajador del campo, oriundo de la alpujarra de Granada."),
    ("human", "Traduce el siguiente texto al {language}: {text}")
])

In [22]:
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce el siguiente texto al {language}: {text}'), additional_kwargs={})])

In [23]:
prompt_template.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={})

In [24]:
prompt_template.messages[1]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce el siguiente texto al {language}: {text}'), additional_kwargs={})

In [25]:
prompt_template.messages[1].prompt.input_variables

['language', 'text']

In [26]:
language = "Granaíno"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

In [27]:
translation_prompt

[SystemMessage(content='Eres un trabajador del campo, oriundo de la alpujarra de Granada.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Traduce el siguiente texto al Granaíno: ¿De dónde vienes?', additional_kwargs={}, response_metadata={})]

In [28]:
print(type(translation_prompt))
print(type(translation_prompt[0]))
print(type(translation_prompt[1]))

<class 'list'>
<class 'langchain_core.messages.system.SystemMessage'>
<class 'langchain_core.messages.human.HumanMessage'>


In [29]:
translator_response = llm.invoke(translation_prompt)

translator_response.content

'¿Ande vienes, majo?'

In [30]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un noble de la realeza española del siglo XV."),
    ("human", "Traduce esto al {language}: {text}")
])

language = "Castellano antiguo"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

customer_response = llm.invoke(translation_prompt)

customer_response.content

'¿De do vienes?'

## 2.3. - Output Parsers

En LangChain, los output parsers son herramientas clave para procesar y estructurar las respuestas que se obtienen de los modelos de lenguaje antes de ser utilizadas en otros pasos del flujo de trabajo. Estos parsers te permiten transformar la salida en bruto de los modelos en un formato más adecuado para tu aplicación, ya sea en forma de texto, datos estructurados o incluso en la ejecución de funciones específicas.

A continuación, una tabla con los output parsers más comunes en LangChain y su uso principal:

| **Output Parser**        | **¿Para qué sirve?**                                                                                                                                                 |
| ------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `JsonOutputParser`       | Convierte la salida del modelo en un objeto JSON (estructura de diccionario o lista). Ideal cuando esperas respuestas estructuradas como JSON o diccionarios.        |
| `RegexOutputParser`      | Extrae información utilizando expresiones regulares. Se usa cuando la respuesta del modelo sigue un patrón específico que se puede identificar con regex.            |
| `StructuredOutputParser` | Permite parsear la salida en estructuras de datos más complejas. Ideal para cuando necesitas que el modelo devuelva datos tabulares o jerárquicos.                   |
| `VariableParser`         | Analiza las respuestas del modelo en busca de valores de variables específicas. Utilizado cuando deseas extraer datos de las respuestas para usarlos en otros pasos. |
| `TextOutputParser`       | Convierte la salida en un texto plano procesable. Ideal cuando la salida del modelo es simplemente texto sin estructura.                                             |
| `SQLOutputParser`        | Convierte la salida del modelo en consultas SQL estructuradas. Es útil si el modelo está generando consultas a bases de datos.                                       |
| `PythonOutputParser`     | Convierte la salida del modelo en código Python ejecutable. Utilizado cuando necesitas generar código a partir de la respuesta del modelo.                           |
| `ActionOutputParser`     | Permite que la salida sea convertida en una acción específica o una ejecución de función. Ideal para flujos de trabajo más dinámicos donde se deben ejecutar tareas. |

In [31]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "muy asequible!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'muy asequible!'}

In [32]:
customer_review = """\
Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:\
soplador de vela, brisa suave, ciudad ventosa y tornado. \
Llegó en dos días, justo a tiempo para el regalo de aniversario de mi esposa. \
Creo que a mi esposa le gustó tanto que se quedó sin palabras. \
Hasta ahora he sido el único que lo ha usado, y lo he estado usando cada dos mañanas para limpiar las hojas de nuestro césped. \
Es ligeramente más caro que los otros sopladores de hojas que hay por ahí, pero creo que vale la pena por las características adicionales.
"""

review_template = """\
Para el siguiente texto, extrae la siguiente información:

gift: ¿Fue el artículo comprado como un regalo para otra persona? \
Responde True si es sí, False si no o si no se sabe.

delivery_days: ¿Cuántos días tardó en llegar el producto? \
Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier frase sobre el valor o precio,\
y devuélvelas como una lista de Python separada por comas.

Formatea la salida como JSON con las siguientes claves:
gift --> boolean
delivery_days --> integer
price_value --> python list

texto: {text}
"""

In [33]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Para el siguiente texto, extrae la siguiente información:\n\ngift: ¿Fue el artículo comprado como un regalo para otra persona? Responde True si es sí, False si no o si no se sabe.\n\ndelivery_days: ¿Cuántos días tardó en llegar el producto? Si esta información no se encuentra, devuelve -1.\n\nprice_value: Extrae cualquier frase sobre el valor o precio,y devuélvelas como una lista de Python separada por comas.\n\nFormatea la salida como JSON con las siguientes claves:\ngift --> boolean\ndelivery_days --> integer\nprice_value --> python list\n\ntexto: {text}\n'), additional_kwargs={})]


In [34]:
prompt_template.messages[0].prompt.input_variables

['text']

In [35]:
messages = prompt_template.format_messages(text=customer_review)

response = llm.invoke(messages)

print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["ligeramente más caro"]
}


In [36]:
type(response.content)

str

In [37]:
# Si intentamos acceder a response como si fuera un diccionario...
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
# Definir los esquemas de respuesta
response_schemas = [
    ResponseSchema(
        name="gift",
        description="True si fue un regalo, False si no."
    ),
    ResponseSchema(
        name="delivery_days",
        description="Número de días que tardó en llegar el producto."
    ),
    ResponseSchema(
        name="price_value",
        description="Frases relacionadas con el valor o precio del producto."
    )
]

# Crear el parser estructurado
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

print(output_parser)
print(output_parser.get_format_instructions())

response_schemas=[ResponseSchema(name='gift', description='True si fue un regalo, False si no.', type='string'), ResponseSchema(name='delivery_days', description='Número de días que tardó en llegar el producto.', type='string'), ResponseSchema(name='price_value', description='Frases relacionadas con el valor o precio del producto.', type='string')]
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // True si fue un regalo, False si no.
	"delivery_days": string  // Número de días que tardó en llegar el producto.
	"price_value": string  // Frases relacionadas con el valor o precio del producto.
}
```


Cuando se utiliza `response_schemas`, LangChain siempre interpreta que los resultados son string. Por eso en `review_template` se indica:

Formatea la salida como JSON con las siguientes claves:
- gift --> boolean
- delivery_days --> integer
- price_value --> python list

In [ ]:
# Construir el prompt con instrucciones de formato
prompt_template = ChatPromptTemplate.from_template(review_template)

formatted_prompt = prompt_template.format(
    text=customer_review,
    format_instructions=output_parser.get_format_instructions()
)

formatted_prompt

'Human: Para el siguiente texto, extrae la siguiente información:\n\ngift: ¿Fue el artículo comprado como un regalo para otra persona? Responde True si es sí, False si no o si no se sabe.\n\ndelivery_days: ¿Cuántos días tardó en llegar el producto? Si esta información no se encuentra, devuelve -1.\n\nprice_value: Extrae cualquier frase sobre el valor o precio,y devuélvelas como una lista de Python separada por comas.\n\nFormatea la salida como JSON con las siguientes claves:\ngift --> boolean\ndelivery_days --> integer\nprice_value --> python list\n\ntexto: Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:soplador de vela, brisa suave, ciudad ventosa y tornado. Llegó en dos días, justo a tiempo para el regalo de aniversario de mi esposa. Creo que a mi esposa le gustó tanto que se quedó sin palabras. Hasta ahora he sido el único que lo ha usado, y lo he estado usando cada dos mañanas para limpiar las hojas de nuestro césped. Es ligeramente más caro que los otro

In [ ]:
# Llamar al modelo
response = llm.invoke([HumanMessage(content=formatted_prompt)])

In [ ]:
# Parsear la salida
parsed_output = output_parser.parse(response.content)

# Mostrar el resultado como diccionario
print(parsed_output)

{'gift': True, 'delivery_days': 2, 'price_value': ['ligeramente más caro']}


In [ ]:
for key, value in parsed_output.items():
    print(f"Clave: {key} | Valor: {value} | Tipo de dato: {type(value)}")

Clave: gift | Valor: True | Tipo de dato: <class 'bool'>
Clave: delivery_days | Valor: 2 | Tipo de dato: <class 'int'>
Clave: price_value | Valor: ['ligeramente más caro'] | Tipo de dato: <class 'list'>


## 2.4. - LangChain: Memory

En LangChain, la memoria se refiere a la capacidad de una cadena o agente para recordar información entre interacciones. Es útil especialmente en contextos conversacionales o multistep, donde necesitas que el modelo tenga conocimiento del contexto anterior.

Por defecto, los modelos como GPT no tienen memoria entre llamadas. LangChain permite gestionar esto incluyendo “memoria” en una cadena (Chain) o un agente (Agent), haciendo que el contexto previo se incluya automáticamente en nuevas solicitudes.

A continuación, una tabla con los tipos de memoria más comunes en LangChain y su uso principal:

| **Tipo de Memoria**              | **¿Para qué sirve?**                                                                                                                                    |
| -------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `ConversationBufferMemory`       | Guarda todo el historial de la conversación como una cadena de texto. Es la forma más simple de mantener el contexto completo de una conversación.      |
| `ConversationBufferWindowMemory` | Similar a `ConversationBufferMemory`, pero solo conserva las **últimas *k* interacciones** (ventana deslizante). Útil para limitar el contexto enviado. |
| `ConversationTokenBufferMemory`  | Guarda la conversación basándose en un **límite de tokens**, no de turnos. Conserva solo la parte del historial que cabe dentro de un número de tokens. |
| `ConversationSummaryMemory`      | Resume el historial de la conversación en un texto corto. Ideal para mantener contexto en conversaciones largas sin exceder el límite de tokens.        |

Diferencias clave:

| Memoria                          | ¿Recuerda todo?       | ¿Controla tamaño? | ¿Resume contenido? | ¿Ideal para...?                               |
| -------------------------------- | --------------------- | ----------------- | ------------------ | --------------------------------------------- |
| `ConversationBufferMemory`       | Sí                    | ❌                 | ❌                  | Conversaciones cortas o demostraciones        |
| `ConversationBufferWindowMemory` | No (solo últimas *k*) | ✅ (*k*)           | ❌                  | Chats donde solo importa el contexto reciente |
| `ConversationTokenBufferMemory`  | No                    | ✅ (*n* tokens)    | ❌                  | Ajustar a límites estrictos de tokens         |
| `ConversationSummaryMemory`      | No (resume todo)      | ✅                 | ✅                  | Chats largos sin perder el hilo               |



In [ ]:
from langchain_openai import ChatOpenAI


from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationTokenBufferMemory,
    ConversationSummaryBufferMemory
)

from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate


A continuación vemos un ejemplo, de como los modelos no incorporan memoria de forma nativa:

In [ ]:
# Instanciar modelo
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

# Primera interacción
message = "Mi nombre es Guille."
response = llm.invoke([HumanMessage(content=message)])
print("Primera respuesta:")
print(response.content)

# Segunda interacción sin memoria
message = "¿Cuál es mi nombre?"
response = llm.invoke([HumanMessage(content=message)])
print("\nSegunda respuesta (sin memoria):")
print(response.content)

Primera respuesta:
¡Hola Guille! ¿En qué puedo ayudarte hoy?

Segunda respuesta (sin memoria):
Lo siento, no tengo la capacidad de saber tu nombre a menos que me lo digas. ¿Cuál es tu nombre?


Vamos a darle memoria:

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

/tmp/ipykernel_224496/1346084784.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_224496/1346084784.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
conversation.predict(input="Hola, mi nombre es Guille")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¿Cómo estás hoy? Soy un asistente de inteligencia artificial aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Hola, mi nombre es Guille
AI:

> Finished chain.


'¡Hola Guille! Me alegra conocerte. ¿Cómo puedo ayudarte hoy?'

In [ ]:
conversation.predict(input="¿Cuánto es 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¿Cómo estás hoy? Soy un asistente de inteligencia artificial aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! Me alegra conocerte. ¿Cómo puedo ayudarte hoy?
Human: ¿Cuánto es 1+1?
AI:

> Finished chain.


'1 + 1 es igual a 2.'

In [ ]:
conversation.predict(input="¿Cuál es mi nombre?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¿Cómo estás hoy? Soy un asistente de inteligencia artificial aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! Me alegra conocerte. ¿Cómo puedo ayudarte hoy?
Human: ¿Cuánto es 1+1?
AI: 1 + 1 es igual a 2.
Human: ¿Cuál es mi nombre?
AI:

> Finished chain.


'Tu nombre es Guille.'

In [ ]:
print(memory.buffer)
print(memory.load_memory_variables({}))

Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?
Human: ¿Cuánto es 1+1?
AI: 1+1 es igual a 2. ¿Hay algo más en lo que pueda ayudarte, Guille?
Human: What is my name?
AI: Tu nombre es Guille. ¿Hay algo más en lo que pueda ayudarte?
{'history': 'Human: Hola, mi nombre es Guille\nAI: ¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?\nHuman: ¿Cuánto es 1+1?\nAI: 1+1 es igual a 2. ¿Hay algo más en lo que pueda ayudarte, Guille?\nHuman: What is my name?\nAI: Tu nombre es Guille. ¿Hay algo más en lo que pueda ayudarte?'}


In [ ]:
memory = ConversationBufferMemory()

memory.save_context(
    {"input": "Hola"}, 
    {"output": "¿Qué tal?"}
)

print(memory.buffer)

Human: Hi
AI: What's up


In [ ]:
memory.save_context(
    {"input": "No mucho, aquí andamos"}, 
    {"output": "Guay"}
)

print(memory.buffer)
print(memory.load_memory_variables({}))

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool
{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}


In [ ]:
# Otras opciones de memoria
# memory = ConversationBufferWindowMemory(k=1)
# memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
# memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

In [ ]:

# Crear la memoria resumida (usa otro LLM para resumir, por defecto GPT-3.5)
summary_memory = ConversationSummaryBufferMemory(
    llm=ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo",
    ),
    max_token_limit=50
)

# LLM para el chat (se puede utilizar el mismo LLM para ambas cosas, o no)
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo"
)

# Conversación con memoria
conversation = ConversationChain(
    llm=llm, 
    memory=summary_memory,
    verbose=True
)

# Interacciones
respuesta1 = conversation.predict(input="Hola, mi nombre es Guille, soy de Madrid y estoy aprendiendo a utilizar LangChain")
print("\n🗨️ Respuesta 1:")
print(respuesta1)

respuesta2 = conversation.predict(input="Trabajo como ingeniero de datos.")
print("\n🗨️ Respuesta 2:")
print(respuesta2)

respuesta3 = conversation.predict(input="¿Qué sabes de mí?")
print("\n🧠 Respuesta 3 (modelo usando resumen):")
print(respuesta3)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, mi nombre es Guille, soy de Madrid y estoy aprendiendo a utilizar LangChain
AI:

> Finished chain.

🗨️ Respuesta 1:
¡Hola Guille! ¡Qué gusto conocerte! LangChain es una plataforma increíble para aprender idiomas. ¿En qué idioma estás interesado en aprender?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as Guille from Madrid and mentions they ar

In [ ]:
print(summary_memory.buffer)
print(summary_memory.load_memory_variables({}))

System: The human introduces themselves as Guille from Madrid and mentions they are learning to use LangChain. The AI greets Guille and praises LangChain as a great platform for learning languages. The AI asks Guille what language they are interested in learning. Guille responds that they work as a data engineer. The AI finds this interesting and asks if Guille is looking to improve their skills in a specific programming language. The AI then greets Guille in Spanish and summarizes what it knows about Guille, including their use of LangChain and their job as a data engineer. The AI also asks if Guille is interested in improving their skills in a specific programming language.
{'history': 'System: The human introduces themselves as Guille from Madrid and mentions they are learning to use LangChain. The AI greets Guille and praises LangChain as a great platform for learning languages. The AI asks Guille what language they are interested in learning. Guille responds that they work as a da

## 2.5. - Chains

En LangChain, una Chain (cadena) es una composición de pasos que conecta modelos de lenguaje con otras herramientas como funciones, bases de datos, prompts, parsers, memoria, etc. En lugar de lanzar una sola petición a un LLM, puedes encadenar operaciones de forma estructurada y reutilizable.

En otras palabras, una Chain permite definir flujos de trabajo con lógica.

A continuación, una tabla resumen de las Chains más comunes en LangChain y su uso principal:

| **Chain**               | **¿Para qué sirve?**                                                                                                                                              |
| ----------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `LLMChain`              | Es la cadena más simple. Conecta un `PromptTemplate` con un LLM. Ideal para tareas de entrada/salida básicas.                                                     |
| `SequentialChain`       | Permite ejecutar varias `Chains` de forma secuencial, pasando la salida de una como entrada de la siguiente. Útil para flujos paso a paso.                        |
| `SimpleSequentialChain` | Variante de `SequentialChain` más sencilla, que solo encadena directamente la salida de un paso como entrada al siguiente (sin nombres de variables intermedias). |
| `RouterChain`           | Redirige automáticamente la entrada a distintas sub-chains según su contenido. Ideal para sistemas multi-agente o flujos condicionados.                           |
| `ConversationChain`     | Incorpora memoria conversacional (como `BufferMemory`). Ideal para construir asistentes o chatbots con contexto.                                                  |
| `TransformChain`        | Aplica una transformación Python entre pasos. Útil para parsear, preprocesar o postprocesar datos entre modelos.                                                  |
| `MapReduceChain`        | Divide una tarea entre varios modelos (Map), luego une las respuestas (Reduce). Útil para resumir o analizar muchos documentos.                                   |
| `RetrievalQAChain`      | Combina un LLM con un buscador de documentos. Extrae contexto relevante antes de preguntar. Ideal para RAG.                                                       |

Las anteriores Chains son desarrolladas por LangChain, pero el usuario también puede definir sus propias chains encadenando diferentes modelos, templates, parsers...etc.

In [ ]:
import pandas as pd

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.router import MultiPromptChain

from langchain_openai import ChatOpenAI


In [ ]:
data = {
    "nombre": ["EcoWave", "NeuroByte", "PetNanny", "Foodloop"],
    "sector": ["energía", "tecnología", "mascotas", "alimentación"]
}

df = pd.DataFrame(data)

df.head()

,nombre,sector
0,EcoWave,energía
1,NeuroByte,tecnología
2,PetNanny,mascotas
3,Foodloop,alimentación


In [ ]:
# Instanciar modelo
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.9
)


### 2.5.1. - LLMChain

In [ ]:
# Chain para generar eslogan
slogan_prompt = PromptTemplate.from_template(
    "Inventa un eslogan creativo para una startup llamada {nombre} del sector {sector}."
)

slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt,
    output_key="eslogan"
)

slogan_chain.invoke({
    "nombre": df[['nombre']],
    "sector": df[['sector']]
})

{'nombre':       nombre
 0    EcoWave
 1  NeuroByte
 2   PetNanny
 3   Foodloop,
 'sector':          sector
 0       energía
 1    tecnología
 2      mascotas
 3  alimentación,
 'eslogan': '1. EcoWave: "Un mar de soluciones sostenibles"\n2. NeuroByte: "Conectando mentes brillantes"\n3. PetNanny: "Cuidando a tus peluditos como si fueran nuestros"\n4. Foodloop: "Tu vuelta al mundo en cada bocado"'}

### 2.5.2. - SimpleSequentialChain

Una SimpleSequentialChain es un conjunto de chains secuenciales, donde cada chain coge como input el output de la chain precedente sin indicar el nombre de los inputs/outputs de forma explícita. 

In [ ]:
# Chain para generar eslogan
slogan_prompt = ChatPromptTemplate.from_template(
    "Inventa un eslogan creativo para una startup llamada {nombre}"
)

slogan_chain = LLMChain(llm=llm, prompt=slogan_prompt)


# Chain para campaña
campaign_prompt = ChatPromptTemplate.from_template(
    "Dado el eslogan {eslogan}, escribe una breve idea para una campaña publicitaria que transmita ese mensaje."
)

campaign_chain = LLMChain(llm=llm, prompt=campaign_prompt)


# Flujo combinado
overall_simple_chain = SimpleSequentialChain(
    chains=[slogan_chain, campaign_chain],
    verbose=True
)

In [ ]:
overall_simple_chain.run(df[:1][['nombre']])

Error in LangChainTracer.on_chain_start callback: ValueError('The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')
Error in LangChainTracer.on_chain_start callback: ValueError('The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')




> Entering new SimpleSequentialChain chain...
"Surfeando hacia un futuro sostenible con EcoWave"
La campaña publicitaria consistiría en mostrar a personas surfistas disfrutando de las olas en un entorno natural y limpio, con un mensaje que invite a cuidar el medio ambiente y a adoptar prácticas sostenibles en nuestro día a día. Se podrían incluir imágenes de playas limpias, animales marinos felices y surfistas comprometidos con la protección del océano. La idea es asociar la práctica del surf con la responsabilidad ambiental y promover un futuro sostenible para las generaciones venideras.

> Finished chain.


'La campaña publicitaria consistiría en mostrar a personas surfistas disfrutando de las olas en un entorno natural y limpio, con un mensaje que invite a cuidar el medio ambiente y a adoptar prácticas sostenibles en nuestro día a día. Se podrían incluir imágenes de playas limpias, animales marinos felices y surfistas comprometidos con la protección del océano. La idea es asociar la práctica del surf con la responsabilidad ambiental y promover un futuro sostenible para las generaciones venideras.'

### 2.5.3. - SequentialChain

Similar a SimpleSequentialChain, pero indicando los nombres de inputs y outputs.

In [ ]:
# Chain para generar eslogan
slogan_prompt = PromptTemplate.from_template(
    "Inventa un eslogan creativo para una startup llamada {nombre} del sector {sector}."
)

slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt,
    output_key="eslogan"
)


# Chain para campaña
campaign_prompt = PromptTemplate.from_template(
    "Dado el eslogan {eslogan}, escribe una breve idea para una campaña publicitaria que transmita ese mensaje."
)

campaign_chain = LLMChain(
    llm=llm,
    prompt=campaign_prompt,
    output_key="campaña"
)

# Flujo combinado
sequential_chain = SequentialChain(
    chains=[slogan_chain, campaign_chain],
    input_variables=["nombre", "sector"],
    output_variables=["eslogan", "campaña"],
    verbose=True
)

In [ ]:
# Lista para guardar resultados
resultados = []

# Aplicar la chain a cada fila
for _, fila in df.iterrows():

    entrada = {
        "nombre": fila["nombre"],
        "sector": fila["sector"]
    }

    salida = sequential_chain.invoke(entrada)
    
    resultados.append({
        **entrada,
        **salida  # esto añade 'eslogan' y 'campaña'
    })

# Nuevo DataFrame con resultados
df_resultado = pd.DataFrame(resultados)



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


In [ ]:
df_resultado.head()

,nombre,sector,eslogan,campaña
0,EcoWave,energía,"""EcoWave: Energía sostenible, olas de cambio p...",La campaña publicitaria podría centrarse en mo...
1,NeuroByte,tecnología,"""NeuroByte: conectando mentes, creando futuro""",La campaña publicitaria consistirá en una seri...
2,PetNanny,mascotas,"""PetNanny: cuidando a tus peludos como si fuer...","Nuestra campaña publicitaria se llama ""Familia..."
3,Foodloop,alimentación,"""¡Dale la vuelta a tus comidas con Foodloop! D...",La campaña publicitaria podría incluir imágene...


## 2.6. - Vector Stores y Retrievers

**Vector Store:**

Un vector store es una base de datos optimizada para almacenar y buscar vectores, que son representaciones numéricas de datos, como texto.

Cuando conviertes texto en vectores usando un modelo de embeddings, esos vectores se pueden almacenar en un vector store. Posteriormente, puedes hacer búsquedas por similitud: das un vector (por ejemplo, el de una pregunta del usuario) y el sistema devuelve los vectores más cercanos, que corresponden a los textos más relevantes.

Es la infraestructura que permite búsquedas semánticas rápidas y eficientes.

**Retrievers:**

Un retriever es un componente que, dado un texto de entrada (como una pregunta), recupera los documentos más relevantes desde alguna fuente. Es una abstracción de LangChain que encapsula la lógica de recuperación, sin preocuparse por cómo están almacenados los datos. El objetivo es simplemente: "dame lo más relevante que tengas sobre esta consulta".

Puede estar respaldado por un vector store, un motor de búsqueda clásico, una API externa, o cualquier otra fuente.

Cuando juntas un vector store con un retriever, lo que haces es usar el vector store como backend para que el retriever recupere documentos basándose en similitud semántica. De este modo se puede implementar un RAG, y darle un conocimiento extra al LLM.

In [38]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch

from IPython.display import display, Markdown

In [39]:
# Loader de LangChain que lee el CSV y convierte cada fila en un documento
file = './data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [42]:
loader

In [43]:
docs = loader.load()
docs[:2]

[Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."),
 Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 1}, page_content=': 1\nname: Recycled 

In [44]:
# Instanciar embedding
embeddings = OpenAIEmbeddings()

# Convertir string a vector mediante un embedding
embed = embeddings.embed_query("Hola mi nombre es Guille")

# Visualizar embedding
print(len(embed))
print(embed[:5])

1536
[-0.045542046427726746, 0.014181733131408691, -0.010796173475682735, -0.014056342653930187, -0.01816917024552822]


In [45]:
# Crear VectorStore en memoria
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

/home/gmachin/.local/share/virtualenvs/exploring-langchain-UOzDUSui/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [46]:
# Query en lenguaje natural
query = "Please suggest a shirt with sunblocking"

# Llamar al VectorStore
docs_result = db.similarity_search(query, k=4)

# Documentos relevantes
print(len(docs_result))

4


In [47]:
docs_result[0]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [48]:
docs_result[1]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 374}, page_content=": 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available.")

In [49]:
# Encadenar todas las respuestas en un string
qdocs = "".join([docs_result[i].page_content for i in range(len(docs_result))])

In [50]:
qdocs

': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.: 374\nname: Men\'s Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ c

In [57]:
# Instancia LLM
llm = ChatOpenAI(
    temperature=0.0,
    model="gpt-3.5-turbo"
    )

# Prompt
prompt = f"""
I will give you a list of product entries. Each one starts with 'name:' and includes a 'description:'.
Extract all shirts with sun protection (UPF/UV) and return a markdown table with:

| Name | Summary of Sun Protection Features |

Here are the entries:

{qdocs}
"""

# Invocar al LLM con el prompt
response = llm.invoke(prompt) 

In [60]:
response.content

"| Name | Summary of Sun Protection Features |\n| --- | --- |\n| Sun Shield Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |\n| Men's Plaid Tropic Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |\n| Men's TropicVibe Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |\n| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |"

In [61]:
display(Markdown(response.content))

| Name | Summary of Sun Protection Features |
| --- | --- |
| Sun Shield Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |

In [62]:
# Convertir VectorStore en un retriever compatible con LangChain
retriever = db.as_retriever()

# RAG
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

query =  """
I will give you a list of product entries. Each one starts with 'name:' and includes a 'description:'.
Extract all shirts with sun protection (UPF/UV) and return a markdown table with:

| Name | Summary of Sun Protection Features |
"""

response = qa_stuff.run(query)

In [65]:
display(Markdown(response))

| Name | Summary of Sun Protection Features |
| --- | --- |
| Sun Shield Shirt by | SPF 50+ sun protection, blocks 98% of harmful rays |
| Men's Tropical Plaid Short-Sleeve Shirt | SPF 50+ sun protection, blocks 98% of harmful rays |
| Men's TropicVibe Shirt, Short-Sleeve | SPF 50+ sun protection, blocks 98% of harmful rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ sun protection, blocks 98% of harmful rays |